In [82]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

service = Service(executable_path='/Applications/Chdriver/chromedriver')
driver = webdriver.Chrome(service=service)
driver.get('https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=United%20States&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0')

scroll_times = 5

title = []
company = []
location = []
posted = []
link = []

for _ in range(scroll_times):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')))
    
    listings = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')

    for listing in listings:
        title.append(listing.find_element(By.CLASS_NAME, 'base-search-card__title').text.strip())
        company.append(listing.find_element(By.CSS_SELECTOR, '.hidden-nested-link').text.strip())
        location.append(listing.find_element(By.CLASS_NAME, 'job-search-card__location').text.strip())
        try:
            posted.append(listing.find_element(By.CLASS_NAME, 'job-search-card__listdate').get_attribute('datetime'))
        except NoSuchElementException:
            posted.append('N/A')        
        link.append(listing.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href'))

driver.quit()

df = pd.DataFrame({
    'Title': title,
    'Company': company,
    'Location': location,
    'Posted': posted,
    'Link': link
})

df['Posted'] = pd.to_datetime(df['Posted'], errors='coerce')
df.sort_values(by='Posted', inplace=True, ascending=False)

df.to_csv('linkedin_jobs.csv', index=False)

In [83]:
df.head()

,Title,Company,Location,Posted,Link
74,Data Analyst,Calculated Hire,Atlanta Metropolitan Area,2024-02-05,https://www.linkedin.com/jobs/view/data-analys...
59,Data Analyst,Calculated Hire,Atlanta Metropolitan Area,2024-02-05,https://www.linkedin.com/jobs/view/data-analys...
44,Data Analyst,Calculated Hire,Atlanta Metropolitan Area,2024-02-05,https://www.linkedin.com/jobs/view/data-analys...
29,Data Analyst,Calculated Hire,Atlanta Metropolitan Area,2024-02-05,https://www.linkedin.com/jobs/view/data-analys...
14,Data Analyst,Calculated Hire,Atlanta Metropolitan Area,2024-02-05,https://www.linkedin.com/jobs/view/data-analys...
